In [ ]:
## install necessary library
## UCI BIO_Science Project
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [ ]:
#Load data Here

data_sample=load_dataset("QuyenAnhDE/Diseases_Symptoms")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [ ]:
updated_data=[{'Name': item['Name'], "Symptoms": item["Symptoms"]} for item in data_sample['train']]

In [ ]:
df=pd.DataFrame(updated_data)

In [ ]:
df.head(5)

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [ ]:
## just extract the symptoms
df["Symptoms"]=df["Symptoms"].apply(lambda x: ",".join(x.split(",")))

In [ ]:
df["Symptoms"]

0      Palpitations, Sweating, Trembling, Shortness o...
1               Hoarseness, Vocal Changes, Vocal Fatigue
2      Short stature, Gonadal dysgenesis, Webbed neck...
3      Absence or undescended testicle(s), empty scro...
4      Nausea, vomiting, abdominal pain, General mala...
                             ...                        
395    Severe abdominal or back pain, blood in urine,...
396    Fragile bones, loss of height over time, back ...
397    Joint pain, stiffness, swelling, fatigue, loss...
398    Frequent urination, Increased thirst, Weight loss
399    Fatigue, Increased hunger, Slow healing of wounds
Name: Symptoms, Length: 400, dtype: object

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split

In [ ]:
if torch.cuda.is_available():
  device=torch.device("cuda")
else:
  try:
    device= torch.device('mps')
  except Exception:
    device=torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
tokenizer=GPT2Tokenizer.from_pretrained("distilgpt2")
model=GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#
#Dataset Preparation

In [ ]:
class LanguageDataset(Dataset):
  def __init__(self,df, tokenizer):
    self.labels=df.columns
    self.data=df.to_dict(orient="records")
    self.tokenizer=tokenizer
    x=self.fittest_max_length(df)
    self.max_length=x

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    x=self.data[idx][self.labels[0]]
    y=self.data[idx][self.labels[1]]
    text=f"{x} | {y}"
    tokens=self.tokenizer.encode_plus(text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
    return tokens

  def fittest_max_length(self,df):
    max_length=max(len(max(df[self.labels[0]],key=len)), len(max(df[self.labels[1]],key=len)))
    x=2
    while x<max_length: x=x*2
    return x


In [ ]:
data_sample=LanguageDataset(df,tokenizer)

In [ ]:
data_sample

In [ ]:
train_size=int(0.8 * len(data_sample))
val_size= len(data_sample)- train_size

train_data, val_data=random_split(data_sample,[train_size,val_size])


In [ ]:
BATCH_SIZE=8

In [ ]:
train_loader=DataLoader(train_data,batch_size=BATCH_SIZE, shuffle=True)
val_loader= DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
num_epochs=8

In [ ]:
batch_size=BATCH_SIZE
model_name='distilGPT2'
gpu=0

In [ ]:
criterion=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer=optim.Adam(model.parameters(), lr=5e-4)

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
results=pd.DataFrame(columns=['epochs', "transformer", "batch_size", "gpu", 'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [ ]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(val_loader)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}")


Training Epoch 1/8 Batch Size: 8, Transformer: distilGPT2: 100%|██████████| 40/40 [00:07<00:00,  5.40it/s, Training Loss=0.475]
Validation Epoch 1/8: 100%|██████████| 10/10 [00:00<00:00, 18.24it/s, Validation Loss=0.606]


Epoch: 1, Validation Loss: 0.6056397557258606


Training Epoch 2/8 Batch Size: 8, Transformer: distilGPT2: 100%|██████████| 40/40 [00:07<00:00,  5.44it/s, Training Loss=0.323]
Validation Epoch 2/8: 100%|██████████| 10/10 [00:00<00:00, 18.38it/s, Validation Loss=0.574]


Epoch: 2, Validation Loss: 0.6294008493423462


Training Epoch 3/8 Batch Size: 8, Transformer: distilGPT2: 100%|██████████| 40/40 [00:07<00:00,  5.38it/s, Training Loss=0.265]
Validation Epoch 3/8: 100%|██████████| 10/10 [00:00<00:00, 18.11it/s, Validation Loss=0.845]


...

Epoch: 6, Validation Loss: 0.7844296097755432


Training Epoch 7/8 Batch Size: 8, Transformer: distilGPT2: 100%|██████████| 40/40 [00:07<00:00,  5.28it/s, Training Loss=0.126]
Validation Epoch 7/8: 100%|██████████| 10/10 [00:00<00:00, 17.65it/s, Validation Loss=0.997]


Epoch: 7, Validation Loss: 0.8115814328193665


Training Epoch 8/8 Batch Size: 8, Transformer: distilGPT2: 100%|██████████| 40/40 [00:07<00:00,  5.23it/s, Training Loss=0.0877]
Validation Epoch 8/8: 100%|██████████| 10/10 [00:00<00:00, 16.76it/s, Validation Loss=0.881]

Epoch: 8, Validation Loss: 0.8563287854194641


In [ ]:
input_str="Kidney Failure"

In [ ]:
input_ids=tokenizer.encode(input_str,return_tensors="pt").to(device)

In [ ]:
input_ids

tensor([[48374,  1681, 25743]], device='cuda:0')

In [ ]:
output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Kidney Failure | Decreased urine output, fluid retention, difficulty emptying the bladder


In [ ]:
torch.save(model, 'DiseaseLM.pt')